# 約維安計畫：羅吉斯迴歸（下）

> 第四十四週

![giphy.com](https://i.giphy.com/media/v1.Y2lkPTc5MGI3NjExcnpqbXE0YjhxbHFpMmM0OXA3YWxqcjR1ZTJmaTF4dTk2cWZmbWQ4NyZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/GSbq5jLhJJWrewuzRG/giphy.gif)

來源：<https://i.giphy.com/media/v1.Y2lkPTc5MGI3NjExcnpqbXE0YjhxbHFpMmM0OXA3YWxqcjR1ZTJmaTF4dTk2cWZmbWQ4NyZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/GSbq5jLhJJWrewuzRG/giphy.gif>

In [1]:
import numpy as np
import pandas as pd

## 羅吉斯迴歸的成本函數

在[約維安計畫：羅吉斯迴歸（上）](https://datainpoint.substack.com/p/week-forty-three-logistic-regression-part-one)一文中我們提到了羅吉斯迴歸類別預測模型的定義、交叉熵（cross entropy）成本函數、Sigmoid 函數以及如何運用梯度遞減（gradient descent）學習演算方法求解權重向量。

\begin{gather}
\text{For each epoch:} \\
w := w - \eta \nabla_w J(w) \; \text{, where } \eta \in \mathbb{R}^+ 
\end{gather}

在這個式子中最重要的一個求解任務就是成本函數 J 關於權重向量的梯度，那麼這個成本函數 J，也就是交叉熵成本函數的偏微該怎麼推導呢？首先我們將成本函數 J 的內容再細寫一次：

\begin{gather}
z = Xw \text{,}\; X \in \mathbb{R^{m \times n}} \text{,} \; w \in \mathbb{R^n}\\
\hat{p} = \sigma(z) \text{, where } \sigma(x) = \frac{1}{1 + e^{-x}} \\
J(\hat{p}) = \frac{1}{m} \left( -ylog(\hat{p}) - (1-y)log(1-\hat{p}) \right) \\
J(w) = \frac{1}{m} \left( -ylog(\sigma(Xw)) - (1-y)log(1-\sigma(Xw)) \right)
\end{gather}

仔細檢查成本函數 J 的組成，我們會發現這是一個鏈結函數（chaining function）的結構，內層有對特徵矩陣和權重向量相乘結果應用 Sigmoid 函數，外層則有對 Sigmoid 函數輸出應用對數函數，若是想順利推導其成本函數的梯度，我們可以預期必須具有幾項先備知識：連鎖法則（chain rule）、對數函數微分以及 Sigmoid 函數微分。

## 連鎖法則

連鎖法則可以用於求解鏈結函數的導數，如果一個鏈結函數 h 是由 g 和 f 兩函數依序作用合成，g 函數先作用，其輸出再成為 f 函數的輸入，那麼 h 函數關於 x 的導數為兩個微分結果的相乘，依序是 h 函數關於 g 函數的微分、g 函數關於 x 的微分。

\begin{gather}
h(x) = f(g(x)) \\
h = f \circ g \\
h'(x) = f'(g(x))g'(x) \\
\frac{dh}{dx} = \frac{d h}{dg}\frac{dg}{dx}
\end{gather}

將連鎖法則應用於成本函數 J 的梯度求解，可以將其寫成兩個偏微分結果的結合，依序是成本函數 J 關於對數函數的偏微分、對數函數關於權重向量的偏微分。接下來我們可以拆開來解這兩個偏微分的部分，最後結合起來得到想要求解的「成本函數 J 關於權重向量的梯度」。

## 對數函數偏微分

關於對數函數偏微分，先備知識（不另外證明的部分）為：

\begin{gather}
\frac{d}{dx}log(x) = \frac{1}{x} \\
\frac{d}{dx}log(1-x) = \frac{-1}{1-x}
\end{gather}

因此我們可以先將成本函數 J 關於權重向量的偏微分推導為成本函數 J 關於對數函數的偏微分形式：

\begin{gather}
J(w) = \frac{1}{m} \left( -ylog(\hat{p}) - (1-y)log(1-\hat{p}) \right)\\
\nabla_w J(w) = \frac{1}{m} \left( -y\frac{1}{\hat{p}} \frac{\partial \hat{p}}{\partial z} - (1-y)\frac{-1}{1-\hat{p}} \frac{\partial(1-\hat{p})}{\partial z} \right)
\end{gather}

完成對數函數偏微分的部分，接下來是 Sigmoid 函數偏微分的部分。

## Sigmoid 函數偏微分

關於 Sigmoid 函數偏微分，先備知識（不另外證明的部分）為：

\begin{align}
\frac{d}{dx}\sigma(x) &= \frac{d}{dx} \left( (1+e^{-x})^{-1} \right) \\
&= \frac{0 \cdot (1+e^{-x})-1 \cdot (e^{-x} \cdot (-1) )}{(1 + e^{-x})^2}\\
&= \frac{e^{-x}}{(1+e^{-x})^2}\\
&= \frac{1-1+e^{-x}}{(1+e^{-x})^2}\\
&= \frac{1+e^{-x}}{(1+e^{-x})^2} - \frac{1}{(1+e^{-x})^2} \\
&= \frac{1}{1+e^{-x}} - \frac{1}{(1+e^{-x})^2} \\
&= \frac{1}{1+e^{-x}} \left( 1 - \frac{1}{1+e^{-x}} \right) \\
&= \sigma(x)\left( 1-\sigma(x) \right)
\end{align}

先備知識中的第二式運用到微分的除法規則、第四式則運用到了一個巧妙的 +1-1 的技巧。

\begin{gather}
h(x) = \frac{f(x)}{g(x)}\\
h'(x) = \frac{f(x)g'(x) - f'(x)g(x)}{g^2(x)}
\end{gather}

因此我們可以接著將成本函數 J 關於權重向量的偏微分推導為最終的形式：

\begin{align}
\nabla_w J(w) &= \frac{1}{m} \left( -y\frac{1}{\hat{p}} \frac{\partial \hat{p}}{\partial z} - (1-y)\frac{-1}{1-\hat{p}} \frac{\partial(1-\hat{p})}{\partial z} \right) \\
&= \frac{1}{m} \left( -y\frac{1}{\hat{p}} \sigma(z)(1-\sigma(z))\sigma'(z) + (1-y)\frac{1}{1-\hat{p}}\sigma(z)(1-\sigma(z))\sigma'(z)\right) \\
&= \frac{1}{m}\sigma'(z) \left( -y\frac{1}{\sigma(z)} \sigma(z)(1-\sigma(z)) + (1-y)\frac{1}{1-\sigma(z)}\sigma(z)(1-\sigma(z))\right) \\
&= \frac{1}{m}\sigma'(z) \left( -y(1-\sigma(z)) + (1-y)\sigma(z) \right) \\
&=\frac{1}{m}\sigma'(z)\left( -y+y\sigma(z) + \sigma(z)-y\sigma(z) \right) \\
&=\frac{1}{m}\sigma'(z)\left(\sigma(z)-y \right)\\
&=\frac{1}{m}X^{\top}\left(\sigma(Xw)-y \right)
\end{align}

我們終於推導出以交叉熵作為成本函數 J 關於權重向量的梯度，最後將羅吉斯迴歸梯度遞減的泛型式表示如下。

\begin{gather}
\text{For each epoch:} \\
w := w - \eta \nabla_w J(w) \; \text{, where } \eta \in \mathbb{R}^+ \\
w := w - \eta \frac{1}{m}X^{\top} \left( \sigma(Xw) - y \right)
\end{gather}

## 以 Numpy 實作羅吉斯迴歸

成功推導出以交叉熵作為成本函數 J 關於權重向量的梯度之後，最後我們寫作一個類別 `LogisticRegression`，以 Python 與 Numpy 模組實作，在給定適當資料集的特徵矩陣與目標向量，能夠迭代優化隨機生成的權重向量，進而生成 h 函數。

In [2]:
class LogisticRegression:
    """
    This class defines the vanilla descent algorithm for logistic regression.
    Args:
        fit_intercept (bool): Whether to add intercept for this model.
    """
    def __init__(self, fit_intercept=True):
        self._fit_intercept = fit_intercept
    def sigmoid(self, X):
        """
        This function returns the Sigmoid output as a probability given certain model weights.
        """
        X_w = np.dot(X, self._w)
        p_hat = 1 / (1 + np.exp(-X_w))
        return p_hat
    def find_gradient(self):
        """
        This function returns the gradient given certain model weights.
        """
        m = self._m
        p_hat = self.sigmoid(self._X_train)
        X_train_T = np.transpose(self._X_train)
        gradient = (1/m) * np.dot(X_train_T, p_hat - self._y_train)
        return gradient
    def cross_entropy(self, epsilon=1e-06):
        """
        This function returns the cross entropy given certain model weights.
        """
        m = self._m
        p_hat = self.sigmoid(self._X_train)
        cost_y1 = -np.dot(self._y_train, np.log(p_hat + epsilon))
        cost_y0 = -np.dot(1 - self._y_train, np.log(1 - p_hat + epsilon))
        cross_entropy = (cost_y1 + cost_y0) / m
        return cross_entropy
    def fit(self, X_train, y_train, epochs=10000, learning_rate=0.001):
        """
        This function uses vanilla gradient descent to solve for weights of this model.
        Args:
            X_train (ndarray): 2d-array for feature matrix of training data.
            y_train (ndarray): 1d-array for target vector of training data.
            epochs (int): The number of iterations to update the model weights.
            learning_rate (float): The learning rate of gradient descent.
        """
        self._X_train = X_train.copy()
        self._y_train = y_train.copy()
        m = self._X_train.shape[0]
        self._m = m
        if self._fit_intercept:
            X0 = np.ones((self._m, 1), dtype=float)
            self._X_train = np.concatenate([X0, self._X_train], axis=1)
        n = self._X_train.shape[1]
        self._w = np.random.rand(n)
        n_prints = 10
        print_iter = epochs // n_prints
        for i in range(epochs):
            cross_entropy = self.cross_entropy()
            gradient = self.find_gradient()
            if i % print_iter == 0:
                print("epoch: {:6} - loss: {:.6f}".format(i, cross_entropy))
            self._w -= learning_rate*gradient
        w_ravel = self._w.ravel().copy()
        self.intercept_ = w_ravel[0]
        self.coef_ = w_ravel[1:].reshape(1, -1)
    def predict_proba(self, X_test):
        """
        This function returns predicted probability with weights of this model.
        Args:
            X_test (ndarray): 2d-array for feature matrix of test data.
        """
        m = X_test.shape[0]
        if self._fit_intercept:
            X0 = np.ones((m, 1), dtype=float)
            self._X_test = np.concatenate([X0, X_test], axis=1)
        p_hat_1 = self.sigmoid(self._X_test).reshape(-1, 1)
        p_hat_0 = 1 - p_hat_1
        proba = np.concatenate([p_hat_0, p_hat_1], axis=1)
        return proba
    def predict(self, X_test):
        """
        This function returns predicted label with weights of this model.
        Args:
            X_test (ndarray): 2d-array for feature matrix of test data.
        """
        proba = self.predict_proba(X_test)
        y_pred = np.argmax(proba, axis=1)
        return y_pred

`LogisticRegression` 類別的使用是以 `fit()` 方法生成權重向量，並以 `predict()` 方法生成預測。我們可以將 `LogisticRegression` 類別應用在 Kaggle 的 Titanic 資料集，藉此來驗證它的可用性。

In [3]:
train = pd.read_csv("https://raw.githubusercontent.com/datainpoint/week-forty-four-logistic-regression-part-two/main/train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/datainpoint/week-forty-four-logistic-regression-part-two/main/test.csv")
X_train = train["Fare"].values.reshape(-1, 1)
y_train = train["Survived"].values
X_test = test["Fare"].values.reshape(-1, 1)
logistic_regression = LogisticRegression()
logistic_regression.fit(X_train, y_train)
y_pred = logistic_regression.predict(X_test)

epoch:      0 - loss: 1.664711
epoch:   1000 - loss: 0.814446
epoch:   2000 - loss: 0.761798
epoch:   3000 - loss: 0.724259
epoch:   4000 - loss: 0.697649
epoch:   5000 - loss: 0.678728
epoch:   6000 - loss: 0.665173
epoch:   7000 - loss: 0.655375
epoch:   8000 - loss: 0.648231
epoch:   9000 - loss: 0.642980


第四十四週約維安計畫：羅吉斯迴歸來到尾聲，希望您也和我一樣期待下一篇文章。對於這篇文章有什麼想法呢？喜歡😻、分享🙌、訂閱📨或者留言🙋‍♂️